In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import ipynbname

_FILE_DIR = ipynbname.path().parent#obtain directory of this file
_PROJ_DIR = _FILE_DIR.parent.parent#obtain main project directory
_DATA_DIR = _PROJ_DIR / "dataset"
_EXTRACT_DIR = _PROJ_DIR / "dataExtracted"

G = nx.read_graphml(str(_EXTRACT_DIR / "THE_MERGED_GRAPH.graphml"))

def metanodes(G, meta_key='meta'):
    return {n : d['country'] for n, d in G.nodes(data=True) if d.get(meta_key, False)}

def printResults(mode):

    # load data
    # --------------
    df_targeted = pd.read_csv(str(_EXTRACT_DIR / f"avg_{mode}_targeted.csv"))
    df_rel_targeted = (df_targeted / df_targeted.iloc[0]).copy()

    df_random = pd.read_csv(str(_EXTRACT_DIR / f"avg_{mode}_random.csv"))
    df_rel_random = (df_random / df_random.iloc[0]).copy()
    # --------------

    # plot amount of countries still above threshold
    # --------------
    countries_count_targeted = []
    for row in df_rel_targeted.iterrows():
        countries_count_targeted.append(sum(np.where(row[1] >= 0.5, 1, 0)))

    countries_count_random = []
    for row in df_rel_random.iterrows():
        countries_count_random.append(sum(np.where(row[1] >= 0.5, 1, 0)))
    
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_figheight(7)
    fig.set_figwidth(16)

    ax1.plot(range(1, len(df_targeted)+1), countries_count_targeted, c= 'orange')
    ax2.plot(range(1, len(df_random)+1), countries_count_random, c= 'darkgreen')

    ax1.grid(alpha= 0.3)
    ax2.grid(alpha= 0.3)

    ax1.set_title(f"Amount of countries above {mode} threshold\nin Iteration i (targeted attacks)")
    ax2.set_title(f"Amount of countries above {mode} threshold\nin Iteration i (random attacks)")
    
    ax1.set_xlabel("Iteration (i)")
    ax2.set_xlabel("Iteration (i)")

    ax1.set_ylabel("Amount of countries")

    ax1.set_xlim(1, max(len(df_targeted)+2, len(df_random)+2))
    ax2.set_xlim(1, max(len(df_targeted)+2, len(df_random)+2))

    ax1.set_ylim(bottom= 0)
    ax2.set_ylim(bottom= 0)

    # --------------

    # metanode - country name dictionary + log files
    # --------------
    meta_dict = metanodes(G)

    dropout_file = _EXTRACT_DIR / f"dropouts_{mode}_targeted.txt"
    edges_file = _EXTRACT_DIR / f"iterations_{mode}_targeted.txt"

    last_iter = len(df_targeted)


    # first and last dropouts information
    # --------------
    first_dropouts = {}
    last_dropouts = []
    x = 5
    with open(dropout_file) as f:
        for i, l in enumerate(f):
            itr, info = l.split(": ")

            _, nb = itr.split()
            nb = int(nb)

            node = f"{info.split()[1]} {info.split()[2]}"

            if i < x:
                first_dropouts[meta_dict[node]] = nb
            
            elif nb == last_iter:
                last_dropouts.append(meta_dict[node])
    
    print(f"\nFirst countries to fall below the {mode} threshold:")
    for k, v in first_dropouts.items():
        print(f"Country {k} - Iteration {v}")

    print(f"\nLast countries to fall below the {mode} threshold:")
    for c in last_dropouts:
        print(f"Country {c} - Iteration {last_iter}")

    # first and last edges removed information
    # --------------
    first_edges = []
    last_edges = []
    y = 5
    with open(edges_file) as f:
        for i, l in enumerate(f):
            if i < y:
                itr, info = l.split(": ")
                info = info.strip("\n").strip("removing").strip()
                first_edges.append(info)
            elif i >= (last_iter-y):
                itr, info = l.split(": ")
                info = info.strip("\n").strip("removing").strip()
                last_edges.append(info)

    print(f"\nFirst removed edges:")
    for e in first_edges:
        print(e)
    
    print(f"\nLast removed edges:")
    for e in last_edges:
        print(e)
    # --------------

printResults("connectivity")

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\Nielson\\Studies\\3rdSemester\\NetworkAnalysis\\UCNA\\dataExtracted\\avg_connectivity_targeted.csv'